## Classification

This week we covered classification using logistic regression. The following is the homework for this week where Carlifornia House pricing dataset is used to predict the median_house_value column by transforming it to a classification task.

In [60]:
# import the libraries

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

## Data preparation

- Select only the features from above and fill in the missing values with 0.
- Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.
- Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.
- Create a new column population_per_household by dividing the column population by the column households from dataframe.

In [13]:
# Fetching the data
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv"

cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']

df = pd.read_csv(url, usecols=cols)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [14]:
# Filling missing values with 0

df.fillna(0)

# Creating column rooms_per_household

df['rooms_per_household'] = df['total_rooms']/df['households']

# Creating column bedrooms_per_room

df['bedrooms_per_room'] = df['total_bedrooms']/df['total_rooms']

# Creating column population_per_household

df['population_per_household'] = df['population']/df['households']

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467


## Question 1

In [15]:
# Most frequent observation for ocean_proximity column

df.ocean_proximity.mode()

0    <1H OCEAN
Name: ocean_proximity, dtype: object

### Mode of ocean_proximity feature: <1H OCEAN

## Question 2
- Create the correlation matrix for the numerical features of your train dataset.
    * In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
- What are the two features that have the biggest correlation in this dataset?

In [16]:
# Selecting numeric columns
numeric = ['float64']
df_num = df.select_dtypes(include=numeric)

# Correlation matrix
df_corr = df_num.corr()
df_corr

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,bedrooms_per_room,population_per_household
longitude,1.000000,-0.924664,-0.108197,0.044568,0.069608,0.099773,0.055310,-0.015176,-0.045967,-0.027540,0.092657,0.002476
latitude,-0.924664,1.000000,0.011173,-0.036100,-0.066983,-0.108785,-0.071035,-0.079809,-0.144160,0.106389,-0.113815,0.002366
housing_median_age,-0.108197,0.011173,1.000000,-0.361262,-0.320451,-0.296244,-0.302916,-0.119034,0.105623,-0.153277,0.136089,0.013191
total_rooms,0.044568,-0.036100,-0.361262,1.000000,0.930380,0.857126,0.918484,0.198050,0.134153,0.133798,-0.187900,-0.024581
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723,0.049686,0.001538,0.084238,-0.028355
population,0.099773,-0.108785,-0.296244,0.857126,0.877747,1.000000,0.907222,0.004834,-0.024650,-0.072213,0.035319,0.069863
households,0.055310,-0.071035,-0.302916,0.918484,0.979728,0.907222,1.000000,0.013033,0.065843,-0.080598,0.065087,-0.027309
median_income,-0.015176,-0.079809,-0.119034,0.198050,-0.007723,0.004834,0.013033,1.000000,0.688075,0.326895,-0.615661,0.018766
median_house_value,-0.045967,-0.144160,0.105623,0.134153,0.049686,-0.024650,0.065843,0.688075,1.000000,0.151948,-0.255880,-0.023737
rooms_per_household,-0.027540,0.106389,-0.153277,0.133798,0.001538,-0.072213,-0.080598,0.326895,0.151948,1.000000,-0.416952,-0.004852


### Features with the biggest correlation: total_bedrooms and households

In [17]:
# Making median_house_value binary by creating a variable above_average which is 1 
# if the median_house_value is above its mean value and 0 otherwise.

mean = df['median_house_value'].mean()

df['above_average'] = np.where(df['median_house_value']>mean,1,0)
        
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,above_average
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556,1
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260,1
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945,1
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467,1


In [18]:
# Splitting the data in train/val/test sets, with 60%/20%/20% distribution.

# train and test split
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

# train and validation split
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [19]:
len(df_train), len(df_test), len(df_val)

(12384, 4128, 4128)

In [20]:
# Resetting the indices 
df_train.reset_index(drop=True)
df_test.reset_index(drop=True)
df_val.reset_index(drop=True)

# Getting the y variables
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

# Removing the target variable
df_train.drop(['median_house_value'], axis=1, inplace=True)
df_val.drop(['median_house_value'], axis=1, inplace=True)
df_test.drop(['median_house_value'], axis=1, inplace=True)

## Question 3

- Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
- What is the value of mutual information?
- Round it to 2 decimal digits using round(score, 2)


In [21]:
mis = mutual_info_score(df_train.ocean_proximity, df_train.above_average)
mis

0.10138385763624205

In [22]:
round(mis, 2)

0.1

### Value of mutual information: 0.1

## Question 4
- Train a logistic regression, including ocean_proximity
- Calculate the accuracy on the validation set

In [24]:
# Removing the binarised target column
df_train.drop(['above_average'], axis=1, inplace=True)
df_val.drop(['above_average'], axis=1, inplace=True)
df_test.drop(['above_average'], axis=1, inplace=True)

In [25]:
# One-hot encoding the categorical feature using DictVectorizer
train_dict = df_train.iloc[:, 0:14].to_dict(orient='records')

# Creating an instance of DictVectorizer
dv = DictVectorizer(sparse=False)

# Fitting and transforming the dictionary
X_train = dv.fit_transform(train_dict)

# Doing the same on validation set (transforming only)
val_dict = df_val.iloc[:, 0:14].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [26]:
# Logistic regression
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [27]:
# model bias and weights

print('Bias term: ', model.intercept_[0])
print('Weights: ', model.coef_[0].round(3))

Bias term:  -0.08540403389228242
Weights:  [ 0.188  0.004  0.036  0.106  0.084  1.214  0.473 -1.705  0.019  0.285
  0.842 -0.002  0.011 -0.02   0.002 -0.   ]


In [32]:
# Predicting on validation set 
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.07847128, 0.16795741, 0.95304104, ..., 0.96103024, 0.85404741,
       0.46588232])

In [39]:
# Getting the accuracy
decision = (y_pred >= 0.5).astype(int)


accuracy = (y_val == decision).mean().round(2)
accuracy

0.84

### Accuracy on the validation dataset: 0.84

## Question 5
- Find the least useful features using feature elimination technique
- Train a model with all the features 
- Exclude a feature from this set, train a model without it, record the accuracy for each model.
- Calculate the difference between the original accuracy and the accuracy without the feature

In [43]:
original_accuracy = (y_val == decision).mean()
original_accuracy

0.8355135658914729

In [45]:
# Excluding a feature and training a model without it 

cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
        'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household']

# list to append accuracies
accs = []

# iterating over the features to get accuracies with each feature
for col in cols:
    
    # making a copy of the datasets
    train_5 = df_train.copy()
    val_5 = df_val.copy()
    
    # removing a feature
    del train_5[col]
    del val_5[col]
    
    # one-hot encoding
    train_dict_5 = train_5.to_dict(orient='records')
    val_dict_5 = val_5.to_dict(orient='records')
    
    X_train_5 = dv.fit_transform(train_dict_5)
    X_val_5 = dv.transform(val_dict_5)
    
    # Fitting the model
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_5, y_train)
    
    # Calculating accuracies with validation set and appending
    y_pred_5 = model.predict_proba(X_val_5)[:, 1]
    decision_5 = (y_pred_5 >= 0.5)
    accuracy = (y_val == decision_5).mean()
    accs.append((col, accuracy))



In [46]:
accs

[('longitude', 0.8328488372093024),
 ('latitude', 0.8343023255813954),
 ('housing_median_age', 0.8316375968992248),
 ('total_rooms', 0.8381782945736435),
 ('total_bedrooms', 0.8357558139534884),
 ('population', 0.8263081395348837),
 ('households', 0.8333333333333334),
 ('median_income', 0.7865794573643411),
 ('ocean_proximity', 0.8204941860465116),
 ('rooms_per_household', 0.8357558139534884),
 ('bedrooms_per_room', 0.8367248062015504),
 ('population_per_household', 0.8362403100775194)]

In [47]:
# Calculating the differences

# list to store the differences
diff = []

for acc in range(len(accs)):
    diff.append((accs[acc][0], original_accuracy - accs[acc][1]))

In [48]:
diff

[('longitude', 0.002664728682170492),
 ('latitude', 0.001211240310077466),
 ('housing_median_age', 0.003875968992248069),
 ('total_rooms', -0.002664728682170603),
 ('total_bedrooms', -0.00024224806201555982),
 ('population', 0.009205426356589164),
 ('households', 0.0021802325581394832),
 ('median_income', 0.04893410852713176),
 ('ocean_proximity', 0.015019379844961267),
 ('rooms_per_household', -0.00024224806201555982),
 ('bedrooms_per_room', -0.001211240310077577),
 ('population_per_household', -0.0007267441860465684)]

In [52]:
# getting the feature with the smallest difference
min_value = min(diff)
min_value

('bedrooms_per_room', -0.001211240310077577)

### bedrooms_per_room has the smallest difference
*This is not among the options given*

## Question 6
- Use ridge regression and the original column, 'median_house_value'
- Calculate the RMSEs on these alpha values: [0, 0.01, 0.1, 1, 10] and get the alpha value that gives the best RMSE

In [54]:
# Splitting the data in train/val/test sets, with 60%/20%/20% distribution.

# train and test split
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

# train and validation split
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [56]:
# Resetting the indices 
df_train.reset_index(drop=True)
df_test.reset_index(drop=True)
df_val.reset_index(drop=True)

# Transforming the target variable
y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

# Removing the target variable
df_train.drop(['median_house_value', 'above_average'], axis=1, inplace=True)
df_val.drop(['median_house_value', 'above_average'], axis=1, inplace=True)
df_test.drop(['median_house_value', 'above_average'], axis=1, inplace=True)

In [58]:
# one-hot encoding
train_dict_6 = df_train.to_dict(orient='records')
val_dict_6 = df_val.to_dict(orient='records')

X_train_6 = dv.fit_transform(train_dict_6)
X_val_6 = dv.transform(val_dict_6)

In [68]:
# Fitting the model
model = Ridge(alpha=10, solver="sag", random_state=42)
model.fit(X_train_6, y_train)


Ridge(alpha=10, random_state=42, solver='sag')

In [69]:
# Calculating the RMSE
y_pred = model.predict(X_val_6)
mse = mean_squared_error(y_val, y_pred)
print('RMSE:', np.sqrt(mse).round(3))

RMSE: 0.524


In [67]:
# Trying out different alpha values
alpha = [0, 0.01, 0.1, 1, 10]

rmse_list = []

for a in alpha:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    model.fit(X_train_6, y_train)
    y_pred = model.predict(X_val_6)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse).round(3)
    rmse_list.append(rmse)
    
    print(a, rmse)

0 0.524
0.01 0.524
0.1 0.524
1 0.524
10 0.524


### All the alpha values give the same RMSEs